## Using Langchain, Groq and OpenAI ofr a terminal chatbot

In this tutorial notebook, 

In [ ]:
## Uncomment below to instal newest versiono f langchain 
# !pip install --upgrade --quiet  langchain langchain-openai langchain-experimental langchainhub python-dotenv langchain-groq

In [6]:
## imports 
import os 
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langchain.tools import  tool
from langchain_community.tools import ShellTool
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor,create_react_agent # To load simple ReAct agent. Reason an act
from langchain import hub
import warnings
warnings. filterwarnings("ignore")




In [7]:
## Load API KEYs
# Defien OPENAI API
#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Load Groq API key
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

## Simplest chatbot 

This is the simples chatbot possible, it takes text and input and generates an output. 

In [18]:
## Load Groq chatbot
llm = ChatGroq(temperature=0.1, model_name="mixtral-8x7b-32768") #gemma-7b-it mixtral-8x7b-32768 or llama2-70b-4096

# ## Load OpenAI chatbot
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)


In [22]:
## Define input and print the response
input = 'Tell me a joke about AI'
response = llm.invoke(input)
print(response.content)

Why don't machines ever laugh at jokes? Because they have a static sense of humor. (I'll show myself out...)


In [23]:
## Define function to get response
def chatbot_response(input):
    response = llm.invoke(input)
    return response.content

#Test the function
print(chatbot_response(input))

Why don't machines ever laugh at jokes? Because they have a static sense of humor. (I'll show myself out...)


### Up to this point we have created a simple chatbot that can only interect throug text and does not have tools to interect with the outside world. When we give the chatbot access to tools we are generating an Agent and they can be very usefull to automate tedious task. Be carefull tho, they can break your computer if handle without care!

Lets build a simple agent that can manage your files using the terminal. 


In [25]:
## Redefine the LLM just for consistency
## Load Groq chatbot
llm = ChatGroq(temperature=0.1, model_name="mixtral-8x7b-32768") #gemma-7b-it mixtral-8x7b-32768 or llama2-70b-4096

# Define tool
shell_tool = ShellTool()

# Define list of tools the LLM is going to use 
tools_list = [shell_tool]

# Get the template prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
print(prompt.template)

## Construct the ReAct agent by defining the llm, tools and prompt template
shell_Agent = create_react_agent(llm=llm,tools=tools_list,prompt=prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=shell_Agent, tools=tools_list, verbose=False)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [26]:
## Define input and get Agent's response
input2 = 'What are the files in the current directory'
response_2 = agent_executor.invoke({"input":str(input2)})
response_2['output']

Executing command:
 ls


'The files in the current directory are README.md and Tutorial_notebook.ipynb.'

In [27]:
## Define function to get agent response
def agent_response(input):
    response = agent_executor.invoke({"input":str(input)})
    return response['output']

#Test the function
print(agent_response(input2))

Executing command:
 ls

(After receiving the output from the terminal)

The files in the current directory are "README.md" and "Tutorial\_notebook.ipynb". The file named "After" does not exist or is not a valid file.


In [28]:
#Test the function asking something that does not require the tool
print(agent_response('Tell me a joke?'))

##NOTE: It fails because its only using the shell tool so its trying to answer the question with the tools

Executing command:
 curl -sL https://www.reddit.com/r/jokes/top.json?limit=100 | python3 -c "
import json,sys
data=json.load(sys.stdin)
print(data[0]['data']['children'][0]['data']['joke'])

Executing command:
 curl -sL "https://www.reddit.com/r/jokes/top.json?limit=100" | python3 -c "
import json,sys
data=json.load(sys.stdin)
print(data[0]['data']['children'][0]['data']['joke'])

Executing command:
 curl -sL "https://www.reddit.com/r/jokes/top.json?limit=100"



KeyboardInterrupt: 

## Define an agent with one of the tools being the chatbot what does not have tools


In [31]:
## Define custom tool using the function to get chatbot response
@tool
def chatbot_response(input):
    """Use Chatbot to answer questions that do not require any tools. Do not perform any action if this tool is used."""
    response = llm.invoke(input)
    return response.content

## Define other tools
## Shell tool
shell_tool = ShellTool()


In [32]:
## FOr consistency redefine all the components of the bot

## Load Groq chatbot
llm = ChatGroq(temperature=0.1, model_name="mixtral-8x7b-32768") #gemma-7b-it mixtral-8x7b-32768 or llama2-70b-4096

# Define list of tools the LLM is going to use 
tools_list = [chatbot_response,shell_tool]

# Get the template prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

## Construct the ReAct agent by defining the llm, tools and prompt template
shell_Agent = create_react_agent(llm=llm,tools=tools_list,prompt=prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=shell_Agent, tools=tools_list, verbose=False)

In [34]:
# Define function to get Assistant response
def assistant_agent_response(input):

    # Run executor to get response 
    response = agent_executor.invoke({"input":str(input)})
    return response['output']

In [35]:
## Define input and print the response
input = 'Tell me a joke about AI'
input2 = 'What are the files in the current directory'

print(assistant_agent_response(input))
print(assistant_agent_response(input2))


Why don't machines ever laugh at jokes? Because they have a statically inferior Comprehension Of Comedic Linguistics, Syntax, and Time, or short, SIFCOLSAT score. (I'm very proud of that one)
Executing command:
 ls
The files in the current directory are README.md and Tutorial\_notebook.ipynb.
